Installing Needed Packages

In [ ]:
!pip  install -U farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install PyArabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 KB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install tashaphyne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 KB 4.9 MB/s eta 0:00:00


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import pandas as pd
import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel, strip_tatweel
from pyarabic.araby import tokenize as araby_tokenize
import re
import string
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
import pyarabic.arabrepr
from tashaphyne.stemming import ArabicLightStemmer
import time

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_csv("CompanyReviews.csv")
df.head()

,Unnamed: 0,review_description,rating,company
0,0,رائع,1,talbat
1,1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1,talbat
2,2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1,talbat
3,3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1,talbat
4,4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1,talbat


In [ ]:
df = df.dropna()

In [ ]:
df.review_description.duplicated().sum()

1042

In [ ]:
df.drop(df[df.review_description.duplicated() == True].index, axis = 0, inplace = True)

<ipython-input-13-872e08e58eac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df[df.review_description.duplicated() == True].index, axis = 0, inplace = True)


In [ ]:
df.review_description.duplicated().sum()

0

In [ ]:
df.review_description.duplicated().isnull().sum()

0

In [ ]:
all_reviews = [] #to be able to apply preprocessing functions correctly & smoothly

In [ ]:
for review in df['review_description']:
  all_reviews.append(review)

In [ ]:
stop_words = stopwords.words('arabic')

In [ ]:
def removing_ar_stopwords(single_review):
    original_words = []
    # Tokenize the sentence to tokens
    words = word_tokenize(single_review) # it works on one word not list
    for word in words:
        if word not in stop_words:
            original_words.append(word)
    filtered_sentence = " ".join(original_words)
    return filtered_sentence

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
import string
def remove_punctuation(single_review):
    return single_review.translate(str.maketrans('', '', string.punctuation))

In [ ]:
from farasa.stemmer import FarasaStemmer
def stemming_Farasa(single_review):
  stemmer = FarasaStemmer()
  stemmed_words = []
  words = word_tokenize(single_review)
  for word in words:
    stemmed_words.append(stemmer.stem(word))

  stemmed_sentence = " ".join(stemmed_words)
  return stemmed_sentence

In [ ]:
#testing the function
stemming_Farasa("قال الشاعر حافظ إبراهيم على لسان اللغة العربية")

/usr/local/lib/python3.9/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100%|██████████| 241M/241M [00:25<00:00, 9.57MiB/s]


'قال شاعر حافظ إبراهيم على لسان لغة عربي'

In [ ]:
from snowballstemmer import stemmer
def stemming_snowball(single_review):
  ar_stemmer = stemmer("arabic")
  stemmed_words = []
  words = word_tokenize(single_review)
  for word in words:
    stemmed_words.append(ar_stemmer.stemWord(word))
  stemmed_sentence = " ".join(stemmed_words)
  return stemmed_sentence

In [ ]:
#testing the function
stemming_snowball("قال الشاعر حافظ إبراهيم على لسان اللغة العربية")

'قال شاعر حافظ ابراهيم علي لسان اللغ عرب'

In [ ]:
from tashaphyne.stemming import ArabicLightStemmer
def stemming_arabiclight(single_review):
  arli_stemmer = ArabicLightStemmer()
  stemmed_words = []
  words = word_tokenize(single_review)
  for word in words:
    stemmed_words.append(arli_stemmer.light_stem(word))
  stemmed_sentence = " ".join(stemmed_words)
  return stemmed_sentence

In [ ]:
#testing the function
stemming_arabiclight("قال الشاعر حافظ إبراهيم على لسان اللغة العربية")

'قال شاعر حافظ إبراهيم على سا لغ عرب'

In [ ]:
def stemming_ISRIStemmer(single_review):

    st = ISRIStemmer()
    stemmend_words = []
    words = word_tokenize(single_review)
    for word in words:
        stemmend_words.append(st.stem(word))
    stemmed_sentence = " ".join(stemmend_words)
    return stemmed_sentence

In [ ]:
#testing
stemming_ISRIStemmer("قال الشاعر حافظ إبراهيم على لسان اللغة العربية")

'قال شعر حفظ راهيم على لسن لغة عرب'

In [ ]:
farasa_reviews = []
snowball_reviews = []
arabic_light_reviews = []
isri_reviews = []

In [ ]:
#apply all preprocessing steps using farasa stemmer
#for unknown reason it takes TOO MUCH time without even finishing
start_time = time.time()
for review in all_reviews:
  filtered_review = remove_punctuation(remove_emoji(removing_ar_stopwords(review)))
  farasa_reviews.append(stemming_Farasa(filtered_review))

farasa_processing_time = time.time() - start_time

KeyboardInterrupt: ignored

In [ ]:
farasa_processing_time = time.time() - start_time

In [ ]:
#apply all preprocessing steps using snowball stemmer
start_time = time.time()
for review in all_reviews:
  filtered_review = remove_punctuation(remove_emoji(removing_ar_stopwords(review)))
  snowball_reviews.append(stemming_snowball(filtered_review))

snowball_processing_time = time.time() - start_time

In [ ]:
#apply all preprocessing steps using Arabic Light stemmer
start_time = time.time()
for review in all_reviews:
  filtered_review = remove_punctuation(remove_emoji(removing_ar_stopwords(review)))
  arabic_light_reviews.append(stemming_arabiclight(filtered_review))

arabic_light_processing_time = time.time() - start_time

In [ ]:
#apply all preprocessing steps using isri stemmer
start_time = time.time()
for review in all_reviews:
  filtered_review = remove_punctuation(remove_emoji(removing_ar_stopwords(review)))
  isri_reviews.append(stemming_ISRIStemmer(filtered_review))

isri_processing_time = time.time() - start_time

In [ ]:
print("Farasa Processing Time", farasa_processing_time)
print("Snowball Processing Time", snowball_processing_time)
print("Arabic Light Processing Time", arabic_light_processing_time)
print("ISRI Processing Time", isri_processing_time)

Farasa Processing Time 474.45932602882385
Snowball Processing Time 62.45466709136963
Arabic Light Processing Time 110.37525582313538
ISRI Processing Time 28.077024698257446


In [ ]:
cleaned_df = pd.DataFrame(isri_reviews, columns=['review'])
cleaned_df.head()

,review
0,رئع
1,رنمج رئع جدا سعد لبه حيج شكل سرع
2,طبق غتح دائ يعط وجد تصل شبك انه الن عند تمم شو...
3,لمذ يمك طلب ماكدونالدز؟
4,رنمج ظهر طعم غلق انه بتك فتح بقل كده كتر شهر


In [ ]:
len(cleaned_df)

39003

In [ ]:
final_df = df.join(cleaned_df)


In [ ]:
len(df)

39003

In [ ]:
len(final_df)

39003

In [ ]:
final_df.dropna()
final_df

,Unnamed: 0,review_description,rating,company,review
0,0,رائع,1,talbat,رئع
1,1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1,talbat,رنمج رئع جدا سعد لبه حيج شكل سرع
2,2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1,talbat,طبق غتح دائ يعط وجد تصل شبك انه الن عند تمم شو...
3,3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1,talbat,لمذ يمك طلب ماكدونالدز؟
4,4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1,talbat,رنمج ظهر طعم غلق انه بتك فتح بقل كده كتر شهر
...,...,...,...,...,...
40041,128,تجربه جيده بس ينقصها عدم اهتمام خدمة العملاء ب...,0,swvl,NaN
40042,129,انا ساكنة بمنطقة الكينج ولا توجد عربيات قبل ال...,-1,swvl,NaN
40043,130,جيد ولكن لماذا لا توجد خطوط كثيره من المريوطيه...,0,swvl,NaN
40044,131,جيدا جدا ...ولكن الاسعار عاليه جدا\n,0,swvl,NaN


In [ ]:
final_df.dropna()

,Unnamed: 0,review_description,rating,company,review
0,0,رائع,1,talbat,رئع
1,1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1,talbat,رنمج رئع جدا سعد لبه حيج شكل سرع
2,2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1,talbat,طبق غتح دائ يعط وجد تصل شبك انه الن عند تمم شو...
3,3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1,talbat,لمذ يمك طلب ماكدونالدز؟
4,4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1,talbat,رنمج ظهر طعم غلق انه بتك فتح بقل كده كتر شهر
...,...,...,...,...,...
38996,38996,لسه حجرب,1,telecom_egypt,متز عمل متز سعر بلغ شوه
38997,38997,كيفية التسجيل في,-1,telecom_egypt,رحل كوس حلة باص مش اد كده يري حدث باص حتي تكون...
38998,38998,ليه مش بيجيلي كود التفعيل وليه لازم احط نجمه و...,-1,telecom_egypt,جرب جيد نقص عدم همم خدم عملاء عمل مره وصل شرك ...
39000,39000,هههههههه محدش بيشكر فيكم ليه كده هل هو افتري م...,-1,telecom_egypt,جيد لمذ وجد خطط كثر ريط يصل حتي ندي رمايه؟ رجو...


In [ ]:
final_df = final_df.drop(['review_description'], axis=1)

KeyError: ignored

In [ ]:
final_df = final_df.dropna()

In [ ]:
final_df

,Unnamed: 0,rating,company,review
0,0,1,talbat,رئع
1,1,1,talbat,رنمج رئع جدا سعد لبه حيج شكل سرع
2,2,-1,talbat,طبق غتح دائ يعط وجد تصل شبك انه الن عند تمم شو...
3,3,-1,talbat,لمذ يمك طلب ماكدونالدز؟
4,4,-1,talbat,رنمج ظهر طعم غلق انه بتك فتح بقل كده كتر شهر
...,...,...,...,...
38996,38996,1,telecom_egypt,متز عمل متز سعر بلغ شوه
38997,38997,-1,telecom_egypt,رحل كوس حلة باص مش اد كده يري حدث باص حتي تكون...
38998,38998,-1,telecom_egypt,جرب جيد نقص عدم همم خدم عملاء عمل مره وصل شرك ...
39000,39000,-1,telecom_egypt,جيد لمذ وجد خطط كثر ريط يصل حتي ندي رمايه؟ رجو...


In [ ]:
from google.colab import drive

#drive.mount('/content/drive')
path = '/content/drive/My Drive/cleaned_review.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  final_df.to_csv(f)

Conclusion

In terms of the processing time, we can order the stemming libraries from the best to the worst as follows:
1- ISRI
2- Snowball
3- Arabic Light

I will not state Farasa in the order as it took TOO MUCH time, although the dataset is not that large.

In [ ]:
%%shell
jupyter nbconvert --to html /content/NLP_Lab_Task01.ipynb